# Compressible Euler Flow of an Ideal Gas (1D)

Let $\Omega = [0,L]\subset \mathbb{R}$ be the spatial domain indicated by the variable $x$, and let $[t_0,t_\text{final}]\subset\mathbb{R}$ be the time domain with variable $t$. We consider the one-dimensional Euler equations for the compressible flow of an ideal gas with periodic boundary conditions.
The state is given by

$$
\begin{aligned}
    \vec{q}_\text{c}(x, t) = \left[\begin{array}{c}
        \rho \\ \rho v \\ \rho e
    \end{array}\right],
\end{aligned}
$$

where $\rho = \rho(x,t)$ is the density $[\frac{\text{kg}}{\text{m}^3}]$, $v = v(x,t)$ is the fluid velocity $[\frac{\text{m}}{\text{s}}]$, and $e = e(x, t)$ is the internal energy per unit mass $[\frac{\text{m}^2}{\text{s}^2}]$.
The state evolves according in time according to the following conservative system of partial differential equations (PDEs):

$$
\tag{1.1}
\begin{aligned}
    \frac{\partial\vec{q}_\text{c}}{\partial t}
    = \frac{\partial}{\partial t} \left[\begin{array}{c}
        \rho \\ \rho v \\ \rho e
    \end{array}\right]
    &= -\frac{\partial}{\partial x}\left[\begin{array}{c}
        \rho v \\ \rho v^2 + p \\ (\rho e + p) v
    \end{array}\right]
    & x &\in\Omega,\quad t\in[t_0,t_\text{final}],
    \\
    \vec{q}_\text{c}(0,t) &= \vec{q}_\text{c}(L,t)
    & t &\in[t_0,t_\text{final}],
    \\
    \vec{q}_\text{c}(x,t_0) &= \vec{q}_{\text{c},0}(x)
    & x &\in \Omega,
\end{aligned}
$$

where $p = p(x,t)$ is the pressure $[\text{Pa}] = [\frac{\text{kg}}{\text{m}\cdot\text{s}^2}]$ and $\vec{q}_{\text{c},0}(x)$ is a given initial condition.
The state variables are related via the ideal gas law

$$
\tag{1.2}
\begin{aligned}
    \rho e = \frac{p}{\gamma - 1} + \frac{1}{2}\rho v^{2},
\end{aligned}
$$

where $\gamma = 1.4$ is the nondimensional heat capacity ratio.

**Important**: Note that the dynamics are nonpolynomially nonlinear with respect to $\rho,$ $\rho v,$ and $\rho e$.

Our objective is to construct a reduced-order model (ROM) which can be solved rapidly to produce approximate solutions to the partial differential equation $(1.1)$ for various choices of the initial condition $\vec{q}_{\text{c},0}$.
We will only use data observed over a limited time interval $[t_0, t_\text{obs}]$ with $t_\text{obs} < t_\text{final}$ to train the ROM, but the ROM will be used to predict the solution for the entire time domain $[t_0, t_\text{final}]$.
Hence, the ROM will be **predictive in time** and **predictive in the initial conditions**.

We make use of the following standard scientific python libraries.
See [demo-with-opinf-package.ipynb](./demo-with-opinf-package.ipynb) for a version of this notebook that uses the [`opinf`](https://willcox-research-group.github.io/rom-operator-inference-Python3/source/index.html) package for model reduction tasks.

In [ ]:
import time
import numpy as np
import scipy.integrate
import scipy.linalg as la
import matplotlib.pyplot as plt

Data generation and visualization routines are defined in the auxiliary file [utils.py](./utils.py).

In [ ]:
import utils

utils.configure_matplotlib(latex_is_installed=True)

## Step 1: Acquire Training Data

To generate data, we first define an equidistant grid $\{x_i\}_{i=0}^{n_x}$ over the one-dimensional spatial domain $\Omega$:

$$
\begin{aligned}
    0 &= x_0 < x_1 < \cdots < x_{n_x-1} < x_{n_x} = L,
    &
    \delta x &= \frac{L}{n_x} = x_{i+1} - x_{i},\quad i=0,\ldots,n_x-1.
\end{aligned}
$$

The spatially discretized state vector collects the values of the three state variables at each point in the spatial domain. Because periodic boundary conditions prescribe $q_\text{c}(x_0,t) = q_\text{c}(x_{n_x},t)$, values at the endpoint $x = x_{n_x} = L$ are not included.

$$
\begin{aligned}
    \mathbf{q}_\text{c}(t)
    = \left[\begin{array}{c}
        \\
        \boldsymbol{\rho}(t)
        \\ \\ \hline \\
        \boldsymbol{\rho}\mathbf{v}(t)
        \\ \\ \hline \\
        \boldsymbol{\rho}\mathbf{e}(t)
        \\ \phantom{.}
    \end{array}\right]
    = \left[\begin{array}{c}
        \\
        \rho(x_{0}, t) \\ \vdots \\ \rho(x_{n_{x}-1}, t)
        \\ \\ \hline \\
        (\rho v)(x_{0}, t) \\ \vdots \\ (\rho v)(x_{n_{x}-1}, t)
        \\ \\ \hline \\
        (\rho e)(x_{0}, t) \\ \vdots \\ (\rho e)(x_{n_{x}-1}, t)
        \\ \phantom{.}
    \end{array}\right]
    \in\mathbb{R}^{3n_x}.
\end{aligned}
$$

Our overall goal is to approximate $\mathbf{q}_\text{c}(t)$ over a time grid, which we take to be an equidistant grid of $n_t$ instances starting at $t = 0$:

$$
\begin{aligned}
    0 &= t_0 < t_1 < \cdots < t_{n_t - 1} = t_\text{final},
    &
    \delta t &= \frac{t_\text{final}}{n_t - 1} = t_{j+1} - t_{j}, \quad j=0,\ldots,n_t-2.
\end{aligned}
$$

### Full-order Model

A full-order model (FOM) for the PDE $(1.1)$ is a system of $n = 3n_x$ ODEs that defines evolution equations for the spatially discretized state $\mathbf{q}_{c}(t)$.
The class [`utils.EulerFiniteDifferenceModel`](./utils.py) implements a FOM by approximating the spatial derivative $\frac{\partial}{\partial x}$ with a first-order backward finite difference approximation,

$$
\begin{aligned}
    \frac{\partial}{\partial x}y(x,t)
    \approx \frac{y(x, t) - y(x - \delta x, t)}{\delta x},
\end{aligned}
$$

for $y = \rho v,$ $\rho v^2 + p,$ and $(\rho e + p) v.$
The resulting system is integrated in time using an adaptive fourth/fifth order Runge–Kutta method.

For this experiment, training states are generated for several different initial conditions, determined by fixing the pressure at $10^5~[\text{Pa}]$ and constructing splines for the density and the velocity.

The next code block loads the experiment data.
Note carefully that we **only load data** – the outputs of the FOM – as opposed to loading the FOM itself.

In [ ]:
data = utils.load_experiment_data()

gamma = data["gamma"]  # Heat capacity ratio.

x = data["x"]  # Spatial domain
dx = x[1] - x[0]  # Spatial step size.

t_all = data["t_all"]  # Full time domain of interest.
dt = t_all[1] - t_all[0]  # Time step size.

t_train = data["t_train"]  # Shorter time domain for training data.
Q_fom = data["training_snapshots"]  # Training states, defined over `t_train`.

print(
    f"\nSpatial domain:\t\t[{x[0]}, {x[-1]}] with {x.size} spatial points",
    f"Spatial step size:\t{dx=:.3f}",
    f"\nFull time domain:\t[{t_all[0]}, {t_all[-1]}] "
    f"with {t_all.size} time instances",
    f"Training time domain:\t[{t_train[0]}, {t_train[-1]}] "
    f"with {t_train.size} time instances",
    f"Time step size:\t\t{dt=:.5f}",
    f"\n{Q_fom.shape[0]} training trajectories "
    f"of {Q_fom.shape[-1]} snapshots each",
    sep="\n",
)

### Sanity Check: Visualize Training Data

Before learning a model from data, it is always a good idea to visualize the data and check that it makes sense physically.

In [ ]:
utils.plot_initial_conditions(x, Q_fom)
plt.show()

In [ ]:
fig, axes = utils.plot_traces(x, t_train, Q_fom[0])
axes[0].set_title("Training trajectory with initial condition 0")
plt.show()

## Step 2: Pre-Process Training Data

The data for this problem requires some pre-processing before applying a dimensionality reduction technique. However, not every problem requires these steps, and the type of preprocessing that is most appropriate for another problem may be different than what is presented here.

### Step 2A: Lift to a Polynomial Form

Operator inference (OpInf) uses training snapshots to construct ROMs with polynomial structure. The FOM for $(1.1)$ is not polynomial with respect to the state $\mathbf{q}_\text{c}(t)$, but by changing to the variables $\vec{q} = (v, p, \zeta)$, where $\zeta = 1/\rho$ is the specific volume $[\text{m}^3/\text{kg}]$, the PDE $(1.1)$ can be expressed equivalently as

$$
\tag{1.3}
\begin{aligned}
    \frac{\partial\vec{q}}{\partial t}
    = \frac{\partial}{\partial t} \left[\begin{array}{c}
        v \\ \\ p \\ \\ \zeta
    \end{array}\right]
    &= \left[\begin{array}{c}
        -v \frac{\partial v}{\partial x} - \zeta\frac{\partial p}{\partial x}
        \\ \\
        -\gamma p \frac{\partial v}{\partial x} - v\frac{\partial p}{\partial x}
        \\ \\
        -v \frac{\partial\zeta}{\partial x} + \zeta\frac{\partial v}{\partial x}
    \end{array}\right],
\end{aligned}
$$

which is quadratic with respect to $\vec{q}$.
This will motivate learning a ROM with a quadratic structure.

The next code block defines the transformation from the original variables to the new variables, as well as the inverse transformation.

In [ ]:
def lift(original_state):
    """Map the conservative variables to the specific volume variables,
    [rho, rho*v, rho*e] -> [v, p, 1/rho].
    """
    rho, rho_v, rho_e = np.split(original_state, 3, axis=0)

    v = rho_v / rho
    p = (gamma - 1) * (rho_e - 0.5 * rho_v * v)  # From the ideal gas law.
    zeta = 1 / rho

    return np.concatenate((v, p, zeta))


def unlift(lifted_state):
    """Map the specific volume variables to the conservative variables,
    [v, p, 1/rho] -> [rho, rho*v, rho*e].
    """
    v, p, zeta = np.split(lifted_state, 3, axis=0)

    rho = 1 / zeta
    rho_v = rho * v
    rho_e = p / (gamma - 1) + 0.5 * rho_v * v  # From the ideal gas law.

    return np.concatenate((rho, rho_v, rho_e))

We now apply the variable transformation to the training snapshots.
Recall that `Q_fom` contains several training trajectories, each of which corresponds to different initial conditions.
It is be helpful for later on to keep track of the trajectories separately instead of concatenating them into a single snapshot matrix.

In [ ]:
Q_fom_lifted = np.array([lift(Q) for Q in Q_fom])
Q_fom_lifted.shape

**Important**: In this example, we start with three state variables and apply a variable transformation to get three different state variables. However, this step is often called "lifting" because it is possible to introduce more variables than we started with through the transformation.

### Step 2B: Center and/or Scale

The variables $\vec{q} = (v, p, \zeta)$ have significantly different characteristic scales, which makes it difficult to equally represent all three variables with a single low-dimensional approximation.

In [ ]:
def data_ranges(snapshots):
    for variable, name in zip(
        np.split(np.hstack(snapshots), 3),
        ("velocity", "pressure", "spec.vol"),
    ):
        print(f"Range({name}) = [{variable.min():.2e}, {variable.max():.2e}]")


data_ranges(Q_fom_lifted)

To address this issue, we non-dimensionalize (rescale) the training data so that each variable has the same characteristic scale.
In this example, we select characteristic scales $\bar{v} = 100~[\text{m}/\text{s}]$ for the velocity and $\bar{p} = 10^5~[\text{Pa}=\text{kg}/\text{ms}^2]$ for the pressure.
A corresponding characteristic scale for specific volume can then be determined from the ideal gas law $(1.2)$:

$$
\begin{aligned}
    \bar{\zeta}
    = \frac{\bar{v}^2}{\bar{p}}
    = \frac{(100)^2~[\text{m}^{2}/\text{s}^{2}]}{10^5~[\text{kg}/\text{ms}^2]}
    = \frac{1}{10}~[\text{m}^3/\text{kg}].
\end{aligned}
$$

New, scaled variables are then defined as

$$
\begin{aligned}
    v' = v / \bar{v},
    \qquad
    p' = p / \bar{p},
    \qquad
    \zeta' = \zeta / \bar{\zeta}.
\end{aligned}
$$

In [ ]:
# Characteristic scales for each variable.
v_bar = 1e2
p_bar = 1e5
z_bar = 1e-1  # = v_bar**2 / p_bar


def scale(snapshots):
    """Nondimensionalize the states [v, p, zeta] -> [v', p', zeta']."""
    v, p, zeta = np.split(snapshots, 3, axis=0)
    return np.concatenate([v / v_bar, p / p_bar, zeta / z_bar], axis=0)


def unscale(scaled_snapshots):
    """Redimensionalize the states [v', p', zeta'] -> [v, p, zeta]."""
    v, p, zeta = np.split(scaled_snapshots, 3, axis=0)
    return np.concatenate([v * v_bar, p * p_bar, zeta * z_bar], axis=0)

In [ ]:
# Rescale the variables in each training trajectory.
Q_fom_scaled = np.array([scale(Q) for Q in Q_fom_lifted])

data_ranges(Q_fom_scaled)

**Important**: There are many ways to preprocess data appropriately for the purpose of learning reduced models.

- A common preprocessing step is to center the data around zero before applying a scaling. Note that this can change the form of the ROM to be learned, see [demo-with-opinf-package.ipynb](./demo-with-opinf-package.ipynb) for more details.
- The scaling applied above is based on a brief dimensional analysis of the lifted variables. In more complex settings, it may be more convenient to apply a data-driven scaling (e.g., dividing each variable by its mean value) or to target a certain range for the scaled variables, usually $[-1, 1]$ or $[0, 1]$.

## Step 3: Reduce Data Dimensionality

We now have discretized, lifted, scaled snapshot data stored in the array `Q_fom_scaled`.
Our goal now is to approximate the preprocessed state $\mathbf{q}(t)\in\mathbb{R}^{n},$ $n = 3n_x,$ with only $r$ degrees of freedom for some small $r \ll n$.
That is, we want a matrix $\mathbf{V}_{\!r}\in\mathbb{R}^{n \times r}$ with orthonormal columns such that $\mathbf{q}(t) \approx \mathbf{V}_{\!r}\hat{\mathbf{q}}(t)$ for some $\hat{\mathbf{q}}(t)\in\mathbb{R}^{r}.$

### Compute a POD Basis

A common choice for $\mathbf{V}_{\!r}$ is the proper orthogonal decomposition (POD) basis, obtained from the singular value decomposition (SVD) of the training data.
Let $\mathbf{Q}\in\mathbb{R}^{n \times k}$ be the matrix of all preprocessed snapshots as columns.
The SVD of $\mathbf{Q}$ is a matrix factorization

$$
\begin{aligned}
    \mathbf{Q} = \boldsymbol{\Phi\Sigma\Psi}^\mathsf{T},
\end{aligned}
$$

where $\boldsymbol{\Phi}\in\mathbb{R}^{n \times n}$ has orthonormal columns, $\boldsymbol{\Sigma}\in\mathbb{R}^{n \times n}$ is diagonal, and $\boldsymbol{\Psi}^\mathsf{T}\in\mathbb{R}^{n \times k}$ has orthonormal rows.
Then $\mathbf{V}_{\!r}$ is defined to be the first $r$ columns of $\boldsymbol{\Phi}$, i.e., the first $r$ principal left singular vectors of the matrix of preprocessed training snapshots.

In [ ]:
Q = np.hstack(Q_fom_scaled)
Phi, sigma, PsiT = la.svd(Q, full_matrices=False)

**Note**: In larger problems, it is common for the dimension of the preprocessed training snapshots to be much larger than the number of training snapshots, i.e., $n \gg k$. In this case, $\boldsymbol{\Phi}\in\mathbb{R}^{n \times k}$, $\boldsymbol{\Sigma}\in\mathbb{R}^{k \times k}$, and $\boldsymbol{\Psi}^\mathsf{T}\in\mathbb{R}^{k \times k}.$ An alternative to the SVD in this scenario is to eigendecompose $\mathbf{Q}^\mathsf{T}\mathbf{Q}\in\mathbb{R}^{k \times k}$ to get the first $k$ singular vectors of $\mathbf{Q}.$ This strategy is known as the _method of snapshots_ in the literature.

### Select the Reduced Dimension

One way to select the reduced dimension $r$ when using POD is to examine the singular values of the training snapshots, the diagonal values of $\boldsymbol{\Sigma}$.


In [ ]:
utils.plot_singular_values(sigma, upto=20)
plt.show()

In this example, we choose $r = 9$ because of the large singular value gap between $r = 9$ and $r = 10$.

<!-- In classical model reduction (e.g., balanced truncation), we never truncate the modes in the middle of a plateau of singular values because the modes in a plateau all have similar physical relevance. -->

In [ ]:
# Definte the basis matrix as dominant left singular vectors.
r = 9
Vr = Phi[:, :r]

# Plot the columns of the basis matrix.
utils.plot_basis_vectors(x, Vr)
plt.show()

Solutions of any ROM that uses the basis matrix $\mathbf{V}_{\!r}$ are restricted to the span of these basis vectors.

### Compress Pre-Processed Data

With a basis $\mathbf{V}_{\!r}\in\mathbb{R}^{n \times r}$ in hand, we compress the training snapshots by finding, for each training snapshot $\mathbf{q}_j\in\mathbb{R}^{n}$, the best latent coordinates $\hat{\mathbf{q}}_j\in\mathbb{R}^{r}$ minimizing the reconstruction error $\|\mathbf{q}_j - \mathbf{V}_{\!r}\hat{\mathbf{q}}_j\|_2$.
Because $\mathbf{V}_{\!r}$ has orthonormal columns, this turns out to be $\hat{\mathbf{q}}_j = \mathbf{V}_{\!r}^\mathsf{T}\mathbf{q}_j$.

In [ ]:
def compress(q):
    """Map (scaled) high-dimensional states to low-dimensional coordinates."""
    return Vr.T @ q


def decompress(qhat):
    """Map low-dimensional coordinates to high-dimensional (scaled) states."""
    return Vr @ qhat

In [ ]:
Q_compressed = np.array([compress(Q) for Q in Q_fom_scaled])

print(f"Scaled FOM snapshots array: {Q_fom_scaled.shape=}")
print(f"Compressed snapshots array: {Q_compressed.shape=}")
print(f"Dimension reduction: n={Vr.shape[0]} -> r={Vr.shape[1]}")

### Sanity Check: Reconstruction Error

The reconstruction error of the training snapshots, defined by

$$
\begin{aligned}
    \sum_{j}\|\mathbf{q}_j - \mathbf{V}_{\!r}\hat{\mathbf{q}}_j\|_2
    = \sum_{j}\|\mathbf{q}_j - \mathbf{V}_{\!r}\mathbf{V}_{\!r}^\mathsf{T}\mathbf{q}_j\|_2,
\end{aligned}
$$

gives a sense for how faithfully this basis can approximate the true state.

In [ ]:
Q_fom_projected = np.array([decompress(Qhat) for Qhat in Q_compressed])
reconstruction_error = la.norm(Q_fom_scaled - Q_fom_projected)
relative_recon_error = reconstruction_error / la.norm(Q_fom_scaled)

print(f"Relative reconstruction error: {relative_recon_error:.2%}")

### Unexample: What if the Data are NOT Scaled?

Computing a basis from *unscaled* data can result in a poor low-dimensional state approximation and issues in the eventual ROM.
The next block computes a POD basis from the lifted but unscaled data.

In [ ]:
# Compute a POD basis from the lifted (but unscaled) data.
Phi2, sigma2, PsiT2 = la.svd(np.hstack(Q_fom_lifted))
Vr_bad = Phi2[:, :r]

# Visualize the resulting basis.
utils.plot_basis_vectors(x, Vr_bad, sharey=False)

Note the drastic difference in scales across the state variables.
Because the characteristic scale of the pressure variable is so much larger than the characteristic scale for velocity and specific volume, the basis vectors for velocity and specific volume are smothered.
The problem is also apparent when examining the reconstruction of each variable.

In [ ]:
# Compress the lifted (but unscaled) data with the corresponding POD basis.
Q_fom_lifted_all = np.hstack(Q_fom_lifted)
Q_compressed_bad = Vr_bad.T @ Q_fom_lifted_all

# Try to reconstruct the lifted data and calculate the error.
Q_projected_bad = Vr_bad @ (Q_compressed_bad)
error_bad_all = la.norm(Q_fom_lifted_all - Q_projected_bad)
relative_error_bad_all = error_bad_all / la.norm(Q_fom_lifted_all)

print(f"Relative recon. error of all variables: {relative_error_bad_all:.2%}")

# Examine the reconstruction error for the individual state variables.
for i, (full_variable, Vr_bad_var) in enumerate(
    zip(
        np.split(Q_fom_lifted_all, 3, axis=0),
        np.split(Vr_bad, 3, axis=0),
    )
):
    var_projected_bad = Vr_bad_var @ Q_compressed_bad
    var_error = la.norm(full_variable - var_projected_bad)
    relative_var_error = var_error / la.norm(full_variable)
    print(
        f"Relative recon. error for variable {i+1}: {relative_var_error:.2%}"
    )

The pressure is represented well, but velocity and specific volume are not.
Plotting the reconstruction shows the extent of the issue.

In [ ]:
# Project one training trajectory with the unscaled basis.
Q_lifted0 = Q_fom_lifted[0]
Q_lifted0_badprojection = Vr_bad @ (Vr_bad.T @ Q_lifted0)

fig, axes = utils.plot_traces(x, t_train, Q_lifted0, lifted=True)
axes[0].set_title("Original lifted data")

fig, axes = utils.plot_traces(x, t_train, Q_lifted0_badprojection, lifted=True)
axes[0].set_title("Reconstruction of lifted data")
plt.show()

## Step 4: Learn Reduced Operators from Data

We now have compressed state data, instances of the reduced state variable $\hat{\mathbf{q}}(t)\in\mathbb{R}^{r}$.
The next step is to learn evolution equations for $\hat{\mathbf{q}}(t)$, i.e., to choose an appropriate $\hat{\mathbf{f}}(t, \hat{\mathbf{q}}(t))$ such that $\frac{\text{d}}{\text{d}t}\hat{\mathbf{q}}(t) \approx \hat{\mathbf{f}}(t, \hat{\mathbf{q}}(t))$.

A FOM based on finite differences for the discretized, lifted, scaled state $\mathbf{q}(t)$ would be quadratic due to the quadratic structure of the PDE $(1.3)$ governing the lifted variables,

$$
\begin{aligned}
    \frac{\text{d}}{\text{d}t}\mathbf{q}(t)
    = \mathbf{H}[\mathbf{q}(t) \otimes \mathbf{q}(t)]
\end{aligned}
$$

for some $\mathbf{H}\in\mathbb{R}^{n \times n^2}$, where $\otimes$ denotes the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product).
Inserting the approximation $\mathbf{q}(t) \approx \mathbf{V}_{\!r}\hat{\mathbf{q}}(t)$ and using the orthogonality property $\mathbf{V}_{\!r}^\mathsf{T}\mathbf{V}_{\!r} = \mathbf{I}$ yields

$$
\tag{1.4}
\begin{aligned}
    \frac{\text{d}}{\text{d}t}\hat{\mathbf{q}}(t)
    = \hat{\mathbf{H}}[\hat{\mathbf{q}}(t) \otimes \hat{\mathbf{q}}(t)]
\end{aligned}
$$

where $\hat{\mathbf{H}} = \mathbf{V}_{\!r}^\mathsf{T}\mathbf{H}[\mathbf{V}_{\!r}\otimes\mathbf{V}_{\!r}] \in \mathbb{R}^{r \times r^2}$.
However, we cannot compute $\hat{\mathbf{H}}$ this way because we do not have $\mathbf{H}$.

The OpInf approach to learning $(1.4)$ is to infer a suitable $\hat{\mathbf{H}}$ by solving the following linear least-squares regression of the compressed training data:

$$
\tag{1.5}
\begin{aligned}
    \min_{\hat{\mathbf{H}}\in\mathbb{R}^{r \times r^2}}
    \sum_{j=1}^{K}\left\|
        \hat{\mathbf{H}}[\hat{\mathbf{q}}_j\otimes\hat{\mathbf{q}}_j]
        - \dot{\hat{\mathbf{q}}}_j
    \right\|_2^2
    + \lambda^2\left\|\hat{\mathbf{H}}\right\|_F^2
\end{aligned}
$$

where the $\hat{\mathbf{q}}_j\in\mathbb{R}^{r}$ are the compressed training snapshots, $\dot{\hat{\mathbf{q}}}_j\in\mathbb{R}^{r}$ are corresponding time derivatives of the snapshots, and $\lambda \ge 0$ is a regularization hyperparameter.

### Construct Data Matrices

The regression $(1.5)$ can be written in standard linear least-squares form as

$$
\begin{aligned}
    \min_{\hat{\mathbf{H}}\in\mathbb{R}^{r\times r^2}}
    \left\|\left(\hat{\mathbf{Q}} \odot \hat{\mathbf{Q}}\right)^\mathsf{T}\hat{\mathbf{H}}^\mathsf{T} - \dot{\hat{\mathbf{Q}}}^\mathsf{T}\right\|_F^2
    + \lambda^2\left\|\hat{\mathbf{H}}\right\|_F^2,
\end{aligned}
$$

where $\hat{\mathbf{Q}}\in\mathbb{R}^{r\times k}$ contains all compressed training snapshots (as columns), $\dot{\hat{\mathbf{Q}}}\in\mathbb{R}^{r\times k}$ contains all corresponding time derivatives, and $\odot$ is the Kronecker product applied columnwise.

To begin setting up this problem, we estimate the time derivatives of the compressed training snapshots using finite differences.
**Having accurate time derivatives is crucial**; poor time derivate estimates will result in a poor ROM.
Because the original FOM was solved with a fourth-order explicit scheme (RK45), we estimate the derivatives with fourth-order forward differences:

$$
\begin{aligned}
    \frac{\text{d}}{\text{d}t}\hat{\mathbf{q}}(t)\bigg|_{t = t_j}
    \approx \frac{1}{12\delta t}(-25\hat{\mathbf{q}}(t_j) + 48\hat{\mathbf{q}}(t_{j} + \delta t) - 36\hat{\mathbf{q}}(t_{j} + 2\delta t)
    + 16\hat{\mathbf{q}}(t_{j} + 3\delta t) - 3\hat{\mathbf{q}}(t_{j} + 4\delta t)).
\end{aligned}
$$

Several snapshots in each trajectory must also be stripped off since the forward finite difference scheme does not provide estimates for the time derivative at the last four time step.

In [ ]:
dt = t_train[1] - t_train[0]


def fwd4(q):
    """Estimate time derivatives with fourth-order forward differences."""
    return (
        -25 * q[:, :-4]
        + 48 * q[:, 1:-3]
        - 36 * q[:, 2:-2]
        + 16 * q[:, 3:-1]
        - 3 * q[:, 4:]
    ) / (12 * dt)


Qhatdot = np.hstack([fwd4(Q) for Q in Q_compressed])
Qhat = np.hstack([Q[:, :-4] for Q in Q_compressed])

print(f"Compressed snapshots array:     {Qhat.shape=}")
print(f"Compressed ddts array:       {Qhatdot.shape=}")

Before constructing the snapshot data matrix $\mathbf{D} = (\hat{\mathbf{Q}}\odot\hat{\mathbf{Q}})^\mathsf{T}\in\mathbb{R}^{k\times r^2}$, note that the Kronecker product $\hat{\mathbf{q}}(t)\otimes\hat{\mathbf{q}}(t)$ has redundant entries (e.g., $\hat{q}_1\hat{q}_2$ and $\hat{q}_2\hat{q}_1$).
As a consequence, $\mathbf{D}$ does not have full column rank and the least-squares problem $(1.5)$ does not have a unique solution when we use the full Kronecker product.

In [ ]:
# Use the full Kronecker product in the data matrix.
D_fullkronecker = la.khatri_rao(Qhat, Qhat).T
D_rank = np.linalg.matrix_rank(D_fullkronecker)

print(f"D.shape = {D_fullkronecker.shape} but rank(D) = {D_rank}")

To remedy the situation, we substitute the full Kronecker product with a _compressed Kronecker product_ that only computes the unique entries of the product.
The compressed product has $r(r+1)/2$ entries instead of $r^2$ entries.

In [ ]:
def compressed_kronecker(state):
    """Unique entries of the Kronecker product of a vector with itself."""
    return np.concatenate(
        [state[i] * state[: i + 1] for i in range(state.shape[0])],
        axis=0,
    )

Now we can form a data matrix $\mathbf{D} = (\hat{\mathbf{Q}}\odot\hat{\mathbf{Q}})^\mathsf{T}\in\mathbb{R}^{k\times r(r+1)/2}$ with full column rank.

In [ ]:
D = compressed_kronecker(Qhat).T

print(f"{D.shape=}, rank(D)={np.linalg.matrix_rank(D)}")
print(f"{Qhatdot.shape=}")

Scaling the data can also improve the condition number of the data matrix.

### Solve the Least-squares Regression

One way to solve $(1.5)$ is to solve the corresponding system of normal equations for $\hat{\mathbf{H}}$,

$$
\begin{aligned}
    \left(\mathbf{D}^\mathsf{T}\mathbf{D} + \lambda^2\mathbf{I}\right)
    \hat{\mathbf{H}}^{\mathsf{T}}
    = \mathbf{D}^\mathsf{T}\dot{\hat{\mathbf{Q}}}^\mathsf{T}.
\end{aligned}
$$

There are other approaches that are more numerically stable, but this way is computationally efficient when tuning the regularization hyperparameter $\lambda$ (see **other example**).
For now, we fix $\lambda = 10^{-8}$.

In [ ]:
# Form the regularization matrix.
regularization = 1e-8
Id = np.eye(D.shape[1])
Reg = regularization**2 * Id

In [ ]:
# Form and solve the normal equations.
Hhat = la.solve(D.T @ D + Reg, D.T @ Qhatdot.T, assume_a="pos").T

# Alternatively, use a least-squares solver (QR, SVD, etc.).
# In this approach, the data matrix and the regularizer must be stacked.
# Hhat2 = la.lstsq(
#     np.vstack([D, Reg]),
#     np.vstack([Qhatdot.T, np.zeros(D.shape[1], Qhatdot.shape[0]))]),
# )[0].T

print(f"{Hhat.shape=}")

### Define the ROM

The resulting ROM is the low-dimensional system of ODEs $(1.4)$ with the $\hat{\mathbf{H}}$ learned through OpInf.

In [ ]:
def _rom_derivative(tt, reduced_state):
    """Right-hand side of the reduced-order model equations."""
    return Hhat @ compressed_kronecker(reduced_state)


def reduced_order_solve(qhat_init, time_domain):
    """Integrate the reduced-order model in time with SciPy."""
    return scipy.integrate.solve_ivp(
        fun=_rom_derivative,
        t_span=[time_domain[0], time_domain[-1]],
        y0=qhat_init,
        method="RK45",
        t_eval=time_domain,
        rtol=1e-6,
        atol=1e-9,
    ).y

## Step 5: Solve the ROM

### Sanity Check: Reproduce Training Data

Before using the ROM for prediction, we use the training initial conditions and compare the ROM solutions to the compressed training data.
This is an important sanity check: if the ROM cannot reproduce the training data, it is unlikely to be able to issue accurate predictions beyond training scenarios.

In [ ]:
print("Relative training errors in the reduced space")

_start = time.time()
for i, Qhat in enumerate(Q_compressed):
    qhat0 = Qhat[:, 0]
    Qhat_rom = reduced_order_solve(qhat0, t_train)
    rom_error = la.norm(Qhat - Qhat_rom) / la.norm(Qhat)
    print(f"  Trajectory {i: >2d}: {rom_error:.4%}")
rom_solve_time = time.time() - _start

print(f"{len(Q_compressed)} ROM solves in {rom_solve_time:.6} s")

In [ ]:
utils.plot_reduced_trajectories(t_train, [Qhat, Qhat_rom])
plt.show()

**Note**: Above, errors are computed with respect to the Frobenius matrix norm, which is what `scipy.linalg.norm()` computes by default.
This is fine for this demonstration, but there are other norms that are more mathematically appropriate for describing distances between time-dependent functions.

### Issue Predictions for New Conditions

Now we take a new initial condition which was **not** used for training, solve the ROM, and compare it with the FOM solution.
We need to remember to do (then undo) all preprocessing steps:
- The new initial condition must be lifted, scaled, and compressed before it is fed to the ROM.
- ROM solutions must be decompressed, unscaled, and unlifted before comparing with the FOM.

In [ ]:
def preprocess(original_state):
    """Map original states to the reduced coordinate space."""
    return compress(scale(lift(original_state)))


def postprocess(reduced_state):
    """Map reduced coordinates to the original state space."""
    return unlift(unscale(decompress(reduced_state)))

In [ ]:
# Map the initial condition to the reduced state space.
_start = time.time()
rom_init = preprocess(data["test_init"])

# Solve the ROM in the reduced state space.
_start2 = time.time()
rom_solution_reduced = reduced_order_solve(rom_init, t_all)
rom_integration_time = time.time() - _start2

# Map the ROM solution back to the original state space.
rom_solution = postprocess(rom_solution_reduced)
rom_solve_time = time.time() - _start

print(f"ROM integrated in {rom_integration_time:.6f} s")
print(f"ROM solve done in {rom_solve_time:.6f} s")

Finally, we test the accuracy of the ROM on this new initial condition.

In [ ]:
fom_solution = data["test_solution"]

# Check the overall ROM error.
test_error = la.norm(fom_solution - rom_solution) / la.norm(fom_solution)
print(f"Overall relative ROM test error: {test_error:.2%}")

# Check the ROM error for each of the physical variables.
for i, (rom_var, fom_var) in enumerate(
    zip(
        np.split(rom_solution, 3, axis=0),
        np.split(fom_solution, 3, axis=0),
    )
):
    test_var_error = la.norm(fom_var - rom_var) / la.norm(fom_var)
    print(f"ROM test error of variable {i+1}: {test_var_error:.2%}")

In [ ]:
utils.plot_regimes(x, t_train, t_all, fom_solution, title="FOM")
utils.plot_regimes(x, t_train, t_all, rom_solution, title="ROM")
plt.show()

See [demo-with-opinf-package.ipynb](./demo-with-opinf-package.ipynb) for a version of this notebook that uses the [`opinf`](https://willcox-research-group.github.io/rom-operator-inference-Python3/source/index.html) package and performs a few additional experiments with this problem.